In [196]:
import re
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate

Creem un dataset a partir del txt que serà un diccionari amb les claus dels idiomes (en i de) i com a valor una llista de les frases traduïdes en ordre

In [171]:
def eliminar_ccby(linea):
    indice_ccby = linea.find("CC-BY")
    if indice_ccby != -1:
        return linea[:indice_ccby]
    return linea

def separar_frases(linea):
    patron = r'(.+?[.!?])\s+([^A-Z]?[\s]+)?(.+)?'
    coincidencias = re.match(patron, linea)
    if coincidencias:
        frase1 = coincidencias.group(1)
        frase2 = coincidencias.group(3)
        return frase1.strip(), frase2.strip() if frase2 else None
    else:
        return None, None

def make_dataset(archivo_entrada):
    dataset = []
    with open(archivo_entrada, 'r') as f_in:
        for linea in f_in:
            linea_clean = eliminar_ccby(linea)
            a,b = separar_frases(linea_clean)
            if a!=None and b!=None:
              dataset.append({'en':a,'de':b})
    return dataset

In [172]:
archivo_entrada = 'deu.txt'
data = make_dataset(archivo_entrada)
len(data)

277227

Tenim 277k traduccions per treballar el model

In [41]:
print(data[:10])

[{'en': 'Go.', 'de': 'Geh.', 'en_tokens': ['<sos>', 'go', '.', '<eos>'], 'de_tokens': ['<sos>', 'geh', '.', '<eos>'], 'en_ids': tensor([ 2, 10,  4,  3]), 'de_ids': tensor([ 2, 27,  4,  3])}, {'en': 'Hi.', 'de': 'Hallo!', 'en_tokens': ['<sos>', 'hi', '.', '<eos>'], 'de_tokens': ['<sos>', 'hallo', '!', '<eos>'], 'en_ids': tensor([  2, 141,   4,   3]), 'de_ids': tensor([ 2, 99,  5,  3])}, {'en': 'Hi.', 'de': 'Grüß Gott!', 'en_tokens': ['<sos>', 'hi', '.', '<eos>'], 'de_tokens': ['<sos>', 'grüß', 'gott', '!', '<eos>'], 'en_ids': tensor([  2, 141,   4,   3]), 'de_ids': tensor([2, 0, 0, 5, 3])}, {'en': 'Run!', 'de': 'Lauf!', 'en_tokens': ['<sos>', 'run', '!', '<eos>'], 'de_tokens': ['<sos>', 'lauf', '!', '<eos>'], 'en_ids': tensor([ 2, 61,  7,  3]), 'de_ids': tensor([  2, 146,   5,   3])}, {'en': 'Run.', 'de': 'Lauf!', 'en_tokens': ['<sos>', 'run', '.', '<eos>'], 'de_tokens': ['<sos>', 'lauf', '!', '<eos>'], 'en_ids': tensor([ 2, 61,  4,  3]), 'de_ids': tensor([  2, 146,   5,   3])}, {'en': 

Importem splitters d'anglés i alemany

In [67]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

In [9]:
string = "What a lovely day it is today!"
[token.text for token in en_nlp.tokenizer(string)]

['What', 'a', 'lovely', 'day', 'it', 'is', 'today', '!']

Para cada par de oraciones añadimos a más su split correspondiente con el incio y fin de frase y tenemos en cuenta las mayúsculas

In [174]:
len(data)

277227

Aquetsa funció només funciona amb un màxim de 1000 elementens del dataset, així que treballarem amb batches per solucionar el problema

In [173]:
def tokenize_example(data, en_nlp, de_nlp, lower, sos_token, eos_token):

    delete = []
    for i in range(len(data)):

        try:
            en_tokens = [token.text for token in en_nlp.tokenizer(data[i]["en"])]
            de_tokens = [token.text for token in de_nlp.tokenizer(data[i]["de"])]
            if lower:
                en_tokens = [token.lower() for token in en_tokens]
                de_tokens = [token.lower() for token in de_tokens]
            en_tokens = [sos_token] + en_tokens + [eos_token]
            de_tokens = [sos_token] + de_tokens + [eos_token]

        except Exception as e:
            delete.append(i)

        else:
            data[i]['en_tokens'] = en_tokens   # només afegirem les dades si no salta cap error, sinó els elements afegits estaran bruts
            data[i]['de_tokens'] = de_tokens

    return data, delete


def process_in_batches(data, batch_size, en_nlp, de_nlp, lower, sos_token, eos_token):

    result = []
    for i in range(0, len(data), batch_size):  # 277 batches de 1000 elements

        batch = data[i:i+batch_size]
        processed_batch, delete = tokenize_example(batch, en_nlp, de_nlp, lower, sos_token, eos_token)
        for frase in delete:
            del(processed_batch[frase])
        result.extend(processed_batch)

    return result

In [175]:
lower = True
sos_token = "<sos>"
eos_token = "<eos>"
batch_size = 1000
dataset = data

inputs = [dataset, batch_size, en_nlp, de_nlp, lower, sos_token, eos_token]

In [176]:
dataset = process_in_batches(*inputs)

In [177]:
len(dataset)

277227

In [178]:
data[1000]

{'en': "It's mine.",
 'de': 'Es ist meins.',
 'en_tokens': ['<sos>', 'it', "'s", 'mine', '.', '<eos>'],
 'de_tokens': ['<sos>', 'es', 'ist', 'meins', '.', '<eos>']}

sos = start of sentence,
eos = end of sentence,
unk = unknown,
pad = padding

In [179]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]


all_en_tokens, all_de_tokens = [], []
for frase in dataset:
    all_en_tokens.append(frase['en_tokens'])
    all_de_tokens.append(frase['de_tokens'])


en_vocab = torchtext.vocab.build_vocab_from_iterator(
    all_en_tokens,
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    all_de_tokens,
    min_freq=min_freq,
    specials=special_tokens,
)

Com a primera conclusió podem dir que l'alemany es un idioma més ric en paraules, ja que es fan servir gairebé el doble en un dataset de 277k frases

In [181]:
words_en, words_de = en_vocab.get_itos(), de_vocab.get_itos()
len(words_en), len(words_de)

(12327, 23055)

In [182]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

In [183]:
"man" in words_en

True

In [184]:
def numericalize_example(data, en_vocab, de_vocab):
    for i in range(len(data)):
        en_ids = en_vocab.lookup_indices(data[i]["en_tokens"])
        de_ids = de_vocab.lookup_indices(data[i]["de_tokens"])
        data[i]['en_ids'] = en_ids
        data[i]['de_ids'] = de_ids
    return data

In [186]:
dataset = numericalize_example(dataset, en_vocab, de_vocab)
dataset[1000]

{'en': "It's mine.",
 'de': 'Es ist meins.',
 'en_tokens': ['<sos>', 'it', "'s", 'mine', '.', '<eos>'],
 'de_tokens': ['<sos>', 'es', 'ist', 'meins', '.', '<eos>'],
 'en_ids': [2, 16, 17, 463, 4, 3],
 'de_ids': [2, 15, 10, 2044, 4, 3]}

L'equivalència token - índex és correcte

In [187]:
print(dataset[100]["en_tokens"])
print(en_vocab.lookup_tokens(dataset[100]["en_ids"]))
dataset[100]["en_tokens"] == en_vocab.lookup_tokens(dataset[100]["en_ids"])

['<sos>', 'no', 'way', '!', '<eos>']
['<sos>', 'no', 'way', '!', '<eos>']


True

In [188]:
for i in range(len(dataset)):
    en_ids = dataset[i]['en_ids']
    de_ids = dataset[i]['de_ids']
    dataset[i]['en_ids'] = torch.tensor(en_ids)
    dataset[i]['de_ids'] = torch.tensor(de_ids)
dataset[100]

{'en': 'No way!',
 'de': 'Das kommt nicht in Frage!',
 'en_tokens': ['<sos>', 'no', 'way', '!', '<eos>'],
 'de_tokens': ['<sos>', 'das', 'kommt', 'nicht', 'in', 'frage', '!', '<eos>'],
 'en_ids': tensor([  2,  77, 150, 188,   3]),
 'de_ids': tensor([  2,  11, 178,   9,  21, 209,  27,   3])}

Ara ja tenim el dataset amb totes les des necessàries per començar el entrenament del model

In [21]:
lista_grande = [i*3 for i in range(100)]

import h5py

# Guardar la lista en un archivo HDF5
with h5py.File('datos.h5', 'w') as f:
    f.create_dataset('lista', data=lista_grande)

# Recuperar la lista del archivo HDF5
with h5py.File('datos.h5', 'r') as f:
    lista_grande_recuperada = f['lista'][:]


lista_grande_recuperada

array([  0,   3,   6,   9,  12,  15,  18,  21,  24,  27,  30,  33,  36,
        39,  42,  45,  48,  51,  54,  57,  60,  63,  66,  69,  72,  75,
        78,  81,  84,  87,  90,  93,  96,  99, 102, 105, 108, 111, 114,
       117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150, 153,
       156, 159, 162, 165, 168, 171, 174, 177, 180, 183, 186, 189, 192,
       195, 198, 201, 204, 207, 210, 213, 216, 219, 222, 225, 228, 231,
       234, 237, 240, 243, 246, 249, 252, 255, 258, 261, 264, 267, 270,
       273, 276, 279, 282, 285, 288, 291, 294, 297])

In [189]:
import torch
import torch.nn as nn

# Supongamos que tenemos un vocabulario de tamaño 10 y queremos vectores de embedding de dimensión 5
vocab_size = 10
embedding_dim = 5

# Creamos una capa de embedding con el tamaño de vocabulario y la dimensión del embedding especificados
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# Creamos una secuencia de entrada de índices
input_indices = torch.tensor([1, 3, 5, 7, 9])

# Pasamos la secuencia de entrada a través de la capa de embedding
embedded_vectors = embedding_layer(input_indices)

# Mostramos los vectores de embedding resultantes
print("Vectores de embedding:")
print(embedded_vectors)

Vectores de embedding:
tensor([[-0.4661,  1.1581,  2.4095,  1.3109, -2.3771],
        [ 0.6151,  0.8339, -0.4458,  1.1323, -0.7849],
        [-0.1015,  0.2155, -0.4441, -0.4906,  0.2156],
        [ 1.9501, -1.8015,  0.3746,  0.8480,  0.8390],
        [ 0.8701, -0.4391,  1.1891, -1.0394,  0.4189]],
       grad_fn=<EmbeddingBackward0>)


Creem una classe básica dicionari per crear el dataloader de la llibreria torch

In [190]:
import torch
from torch.utils.data import Dataset

class ListDictDataset(Dataset):
    def __init__(self, list_of_dicts):
        self.list_of_dicts = list_of_dicts

    def __len__(self):
        return len(self.list_of_dicts)

    def __getitem__(self, index):
        return self.list_of_dicts[index]

#dataset = ListDictDataset(train)
type(dataset)

list

Creem un dataloader per processar bé totes les traduccions, on només afegirem els index de les paraules al diccionari corresponent segons l idioma, que és el que ens servirà per fer l embedding de les frases

In [191]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)  #Fem servir padding com relleno
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

In [192]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

Hem de tenir en compte també que el dataset, tal i com l'hem importat, està ordenat de frases més simples, d'una sola a paraules, a més complexes. Per tant hem de considerar què és millor per entrenar i validad, respectivament

In [200]:
batch_size = 128
test_size = 0.1

random.shuffle(dataset)
test = int( len(dataset) * test_size )
test_set = dataset[:test]
train_set = dataset[-test:]

train_data_loader = get_data_loader(train_set, batch_size, pad_index, shuffle=True)
test_data_loader = get_data_loader(test_set, batch_size, pad_index, shuffle=True)

In [201]:
train_data_loader

Cla

In [28]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, encoder_hidden_dim, bidirectional=True)
        self.fc = nn.Linear(encoder_hidden_dim * 2, decoder_hidden_dim)
        self.dropout = nn.Dropout(dropout)  # recordem: dropout desactiva un % de neurones i així evita l'overfitting

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, hidden = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer
        # hidden [-2, :, : ] is the last of the forwards RNN
        # hidden [-1, :, : ] is the last of the backwards RNN
        # initial decoder hidden is final hidden state of the forwards and backwards
        # encoder RNNs fed through a linear layer
        hidden = torch.tanh(
            self.fc(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        )
        # outputs = [src length, batch size, encoder hidden dim * 2]
        # hidden = [batch size, decoder hidden dim]
        return outputs, hidden

In [29]:
class Attention(nn.Module):
    def __init__(self, encoder_hidden_dim, decoder_hidden_dim):
        super().__init__()
        self.attn_fc = nn.Linear(
            (encoder_hidden_dim * 2) + decoder_hidden_dim, decoder_hidden_dim
        )
        self.v_fc = nn.Linear(decoder_hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden = [batch size, decoder hidden dim]
        # encoder_outputs = [src length, batch size, encoder hidden dim * 2]
        batch_size = encoder_outputs.shape[1]
        src_length = encoder_outputs.shape[0]
        # repeat decoder hidden state src_length times
        hidden = hidden.unsqueeze(1).repeat(1, src_length, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # hidden = [batch size, src length, decoder hidden dim]
        # encoder_outputs = [batch size, src length, encoder hidden dim * 2]
        energy = torch.tanh(self.attn_fc(torch.cat((hidden, encoder_outputs), dim=2)))
        # energy = [batch size, src length, decoder hidden dim]
        attention = self.v_fc(energy).squeeze(2)
        # attention = [batch size, src length]
        return torch.softmax(attention, dim=1)

In [30]:
class Decoder(nn.Module):
    def __init__(
        self,
        output_dim,
        embedding_dim,
        encoder_hidden_dim,
        decoder_hidden_dim,
        dropout,
        attention,
    ):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.GRU((encoder_hidden_dim * 2) + embedding_dim, decoder_hidden_dim)
        self.fc_out = nn.Linear(
            (encoder_hidden_dim * 2) + decoder_hidden_dim + embedding_dim, output_dim
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        # input = [batch size]
        # hidden = [batch size, decoder hidden dim]
        # encoder_outputs = [src length, batch size, encoder hidden dim * 2]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        a = self.attention(hidden, encoder_outputs)
        # a = [batch size, src length]
        a = a.unsqueeze(1)
        # a = [batch size, 1, src length]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # encoder_outputs = [batch size, src length, encoder hidden dim * 2]
        weighted = torch.bmm(a, encoder_outputs)
        # weighted = [batch size, 1, encoder hidden dim * 2]
        weighted = weighted.permute(1, 0, 2)
        # weighted = [1, batch size, encoder hidden dim * 2]
        rnn_input = torch.cat((embedded, weighted), dim=2)
        # rnn_input = [1, batch size, (encoder hidden dim * 2) + embedding dim]
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        # output = [seq length, batch size, decoder hid dim * n directions]
        # hidden = [n layers * n directions, batch size, decoder hid dim]
        # seq len, n layers and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, decoder hidden dim]
        # hidden = [1, batch size, decoder hidden dim]
        # this also means that output == hidden
        assert (output == hidden).all()
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))
        # prediction = [batch size, output dim]
        return prediction, hidden.squeeze(0), a.squeeze(1)

In [31]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        batch_size = src.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # encoder_outputs is all hidden states of the input sequence, back and forwards
        # hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
        # outputs = [src length, batch size, encoder hidden dim * 2]
        # hidden = [batch size, decoder hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden state and all encoder hidden states
            # receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, decoder hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

Valors amb el que anirem probant per millorar l'eficiència del model

In [195]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
encoder_hidden_dim = 512
decoder_hidden_dim = 512
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

attention = Attention(encoder_hidden_dim, decoder_hidden_dim)

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    decoder_dropout,
    attention,
)

model = Seq2Seq(encoder, decoder, device).to(device)

Cada paràmetre representa el weight o bias d'una capa (ih=input, hh=hidden) en el recorregut d'anada per calcular el loss o el de tornada que actualitza el gradient (reverse)

In [33]:
for p in model.named_parameters(): print(p)

('encoder.embedding.weight', Parameter containing:
tensor([[ 0.1174,  0.8136, -1.4527,  ...,  0.5432, -1.2766, -0.1058],
        [-0.3804,  0.4201,  1.5162,  ..., -0.2074,  1.5256,  0.3430],
        [ 0.0426,  1.1837,  0.7930,  ..., -0.7549,  0.5801, -0.5989],
        ...,
        [ 1.5370,  0.9916, -0.5240,  ...,  1.1633,  0.5648,  2.0671],
        [ 0.4614, -3.2469,  1.7588,  ..., -2.3616, -2.0416, -0.2437],
        [-0.7517,  0.4220, -0.4800,  ...,  0.4197, -0.7090,  0.9285]],
       requires_grad=True))
('encoder.rnn.weight_ih_l0', Parameter containing:
tensor([[-0.0034,  0.0057, -0.0421,  ..., -0.0142, -0.0294,  0.0235],
        [-0.0072, -0.0429,  0.0325,  ...,  0.0328,  0.0342, -0.0324],
        [ 0.0206, -0.0045,  0.0392,  ..., -0.0037,  0.0179, -0.0102],
        ...,
        [ 0.0037,  0.0174,  0.0044,  ..., -0.0064, -0.0382, -0.0150],
        [-0.0107, -0.0165, -0.0249,  ...,  0.0022, -0.0416,  0.0121],
        [-0.0082, -0.0133,  0.0198,  ...,  0.0324,  0.0163, -0.0056]],
  

In [34]:
def init_weights(m):
    for name, param in m.named_parameters():
        if "weight" in name:
            nn.init.normal_(param.data, mean=0, std=0.01)  # pesos molt propers a 0, desviació del 1%, perill de vanishing
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(297, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn_fc): Linear(in_features=1536, out_features=512, bias=True)
      (v_fc): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(262, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=262, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [35]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad),model.parameters()

a,b = count_parameters(model)
suma=0
for x in b:
  print(x)

print(f"The model has {a:,} trainable parameters",suma)

Parameter containing:
tensor([[-0.0085, -0.0004, -0.0030,  ...,  0.0078,  0.0084, -0.0094],
        [ 0.0192,  0.0050,  0.0254,  ..., -0.0014,  0.0092, -0.0026],
        [-0.0044, -0.0076, -0.0008,  ...,  0.0082, -0.0071,  0.0024],
        ...,
        [-0.0149,  0.0071, -0.0196,  ...,  0.0175, -0.0188,  0.0053],
        [ 0.0052,  0.0091, -0.0115,  ..., -0.0025, -0.0184, -0.0085],
        [ 0.0070, -0.0065,  0.0121,  ..., -0.0202, -0.0021, -0.0098]],
       requires_grad=True)
Parameter containing:
tensor([[-0.0007,  0.0010,  0.0011,  ..., -0.0107, -0.0123,  0.0054],
        [-0.0009,  0.0108, -0.0045,  ..., -0.0072, -0.0183,  0.0102],
        [ 0.0143,  0.0048,  0.0029,  ..., -0.0119, -0.0026,  0.0012],
        ...,
        [-0.0059,  0.0035,  0.0069,  ...,  0.0047, -0.0054, -0.0093],
        [-0.0008,  0.0182, -0.0003,  ...,  0.0072, -0.0098,  0.0094],
        [-0.0046, -0.0193,  0.0087,  ..., -0.0091, -0.0036, -0.0160]],
       requires_grad=True)
Parameter containing:
tensor([[-0.

In [36]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [37]:
for i in range(len(train)):
  lista = train[i]['de_ids']
  print(lista)

tensor([ 2, 27,  4,  3])
tensor([ 2, 99,  5,  3])
tensor([2, 0, 0, 5, 3])
tensor([  2, 146,   5,   3])
tensor([  2, 146,   5,   3])
tensor([2, 0, 5, 3])
tensor([2, 0, 5, 3])
tensor([  2, 240, 112,   5,   3])
tensor([  2, 132,   5,   3])
tensor([2, 0, 5, 3])
tensor([ 2, 25,  0,  5,  3])
tensor([ 2,  0, 12,  5,  3])
tensor([  2, 178,  50,   5,   3])
tensor([  2, 194,   5,   3])
tensor([2, 0, 5, 3])
tensor([2, 0, 5, 3])
tensor([ 2, 88,  5,  3])
tensor([ 2, 88,  4,  3])
tensor([ 2, 97, 35,  4,  3])
tensor([  2, 247,   7,   5,   3])
tensor([  2, 279,   7,   4,   3])
tensor([  2,  17, 185,   4,   3])
tensor([ 2, 99,  5,  3])
tensor([2, 0, 5, 3])
tensor([ 2, 99,  5,  3])
tensor([  2, 119,  12,   5,   3])
tensor([  2, 264,   5,   3])
tensor([  2,   6, 282,  18,   4,   3])
tensor([  2,   6,  14,  18, 178,   4,   3])
tensor([  2,   6, 110,   4,   3])
tensor([  2,   6, 179,   4,   3])
tensor([  2, 186,   4,   3])
tensor([2, 6, 0, 7, 4, 3])
tensor([2, 6, 0, 7, 4, 3])
tensor([  2,   6, 226,  37,   

Totes les primeres files de cada batch estan plenes de 2, com l'inici de tots els en_ids o de_ids, que representa el eos

In [38]:
loader = train_data_loader
a,b = 0,0

for batch in loader:
  print(batch['en_ids'])

num_batches = len(loader)
print(f'Número de batches: {num_batches}')  #número de batches dins del loader

tensor([[  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
           2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
           2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
           2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
           2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
           2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
           2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
           2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
           2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
           2,   2],
        [113,   5,   5,  10,   5,   5,   9,  49,   5, 111,  26,  41,   9,   5,
          26,   5,  28,  10,  62,  94,  10,   5,   5,  18,   6,  23, 252, 100,
           9,  38,  11, 203,   5,  35,   5,  23, 144,  11,  24,  26,  61,   6,
          32, 149,  28,   0,   5

In [39]:
def train_seq2seq(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["de_ids"].to(device)   # device és gpu collab ja que és on fem les probes per no gasta hores de mv azure
        trg = batch["en_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

def evaluate_seq2seq(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

Ara hem de dividir el conjunt de dades que hem processat en train, test per entrenar i validar correctament la xarxa neuronal

In [202]:
n_epochs = 2
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):  #tqdm per veure el processament del bucle, ja que tardará molt
    train_loss = train_seq2seq(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_seq2seq(
        model,
        test_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut3-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

  0%|          | 0/2 [01:00<?, ?it/s]


KeyboardInterrupt: 